<a href="https://colab.research.google.com/github/Fuohna/nlp-simple-chatbot-v1/blob/main/simple_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U "tensorflow-text==2.8.*"
!pip install -q -U "tensorflow==2.8.*"

In [ ]:
import pandas as pd
import json
import string
import tensorflow as tf
from tensorflow import keras
import tensorflow_text as tf_text
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Preprocessing

In [ ]:
leyzer_dev_url = 'https://raw.githubusercontent.com/cartesinus/leyzer/master/corpora/0.1.0/leyzer-dev-en-US-0.1.0.tsv'
leyzer_dev_df = pd.DataFrame(pd.read_csv(leyzer_dev_url, sep='\t'))

In [ ]:
leyzer_df = leyzer_dev_df
leyzer_df.drop(["bio"], axis = 1, inplace = True)
leyzer_df

,domain,intent,utterance
0,Airconditioner,TurnOff,switch the air conditioning off
1,Airconditioner,TurnOff,turn the ac off
2,Airconditioner,TurnOn,switch on the aircon
3,Airconditioner,TurnOn,turn the aircon on
4,Airconditioner,ChangeTemperature,change the minimum and maximum temperatures on...
...,...,...,...
373,Youtube,FindQueryOnChannel,show a video from niki and gabi matching seinf...
374,Youtube,FindQueryOnChannel,show me a video from cineamasin matching lipst...
375,Youtube,FindQueryOnChannel,show me a videos published by wwe matching gre...
376,Youtube,NotifyOnNewFromFollowing,notify me when there is new video from youtube...


In [ ]:
leyzer_df['utterance'] = leyzer_df['utterance'].apply(lambda sequence: [ltrs.lower() for ltrs in sequence if ltrs not in string.punctuation])
leyzer_df

,domain,intent,utterance
0,Airconditioner,TurnOff,"[s, w, i, t, c, h, , t, h, e, , a, i, r, , ..."
1,Airconditioner,TurnOff,"[t, u, r, n, , t, h, e, , a, c, , o, f, f]"
2,Airconditioner,TurnOn,"[s, w, i, t, c, h, , o, n, , t, h, e, , a, ..."
3,Airconditioner,TurnOn,"[t, u, r, n, , t, h, e, , a, i, r, c, o, n, ..."
4,Airconditioner,ChangeTemperature,"[c, h, a, n, g, e, , t, h, e, , m, i, n, i, ..."
...,...,...,...
373,Youtube,FindQueryOnChannel,"[s, h, o, w, , a, , v, i, d, e, o, , f, r, ..."
374,Youtube,FindQueryOnChannel,"[s, h, o, w, , m, e, , a, , v, i, d, e, o, ..."
375,Youtube,FindQueryOnChannel,"[s, h, o, w, , m, e, , a, , v, i, d, e, o, ..."
376,Youtube,NotifyOnNewFromFollowing,"[n, o, t, i, f, y, , m, e, , w, h, e, n, , ..."


In [ ]:
leyzer_df['utterance'] = leyzer_df['utterance'].apply(lambda word: ''.join(word))
leyzer_df

,domain,intent,utterance
0,Airconditioner,TurnOff,switch the air conditioning off
1,Airconditioner,TurnOff,turn the ac off
2,Airconditioner,TurnOn,switch on the aircon
3,Airconditioner,TurnOn,turn the aircon on
4,Airconditioner,ChangeTemperature,change the minimum and maximum temperatures on...
...,...,...,...
373,Youtube,FindQueryOnChannel,show a video from niki and gabi matching seinf...
374,Youtube,FindQueryOnChannel,show me a video from cineamasin matching lipst...
375,Youtube,FindQueryOnChannel,show me a videos published by wwe matching gre...
376,Youtube,NotifyOnNewFromFollowing,notify me when there is new video from youtube...


In [ ]:
from nltk.tokenize import word_tokenize 
input_tokenized = list(leyzer_df['utterance'].apply(lambda sent: word_tokenize(sent)))
print(input_tokenized)

[['switch', 'the', 'air', 'conditioning', 'off'], ['turn', 'the', 'ac', 'off'], ['switch', 'on', 'the', 'aircon'], ['turn', 'the', 'aircon', 'on'], ['change', 'the', 'minimum', 'and', 'maximum', 'temperatures', 'on', 'my', 'thermostat'], ['set', 'set', 'my', 'thermostat', 'to', '25', 'degree'], ['set', 'the', 'temperature', 'to', '80', 'degrees', 'fahrenheit', 'on', 'my', 'hall', 'thermostat'], ['set', 'the', 'low', 'temperature', 'to', '23', 'degree', 'on', 'my', 'thermostat', 'and', 'the', 'high', 'to', '29', 'degree'], ['change', 'the', 'minimum', 'and', 'maximum', 'temperatures', 'on', 'my', 'living', 'room', 'thermostat'], ['show', 'the', 'humidity', 'level'], ['check', 'the', 'humidity', 'level'], ['what', 'temperature', 'do', 'i', 'have', 'in', 'here', 'according', 'to', 'my', 'cooling', 'system'], ['what', 'temperature', 'is', 'it', 'in', 'here', 'according', 'to', 'my', 'air', 'conditioning'], ['check', 'the', 'temperature', 'of', 'the', 'kitchen', 'room', 'temperature', 'sens

In [ ]:
col = leyzer_df[['intent']]
print("Total number of intents:", len(col.intent.unique()))
# print(col.intent.unique())

Total number of intents: 178


In [ ]:
def my_token(words):
  token = Tokenizer(1000)
  token.fit_on_texts(words)
  return token

In [155]:
tokenizer = Tokenizer(1000)
tokenizer.fit_on_texts(input_tokenized)
train_utr = tokenizer.texts_to_sequences(input_tokenized)
# train_utr = my_token(input_tokenized)
print(train_utr)
features = pad_sequences(train_utr, maxlen = 100)
print(features[377])
print(len(features[0]))

[[118, 4, 173, 174, 119], [120, 4, 277, 119], [118, 1, 4, 175], [120, 4, 175, 1], [56, 4, 176, 14, 177, 178, 1, 3, 101], [57, 57, 3, 101, 2, 278, 143], [57, 4, 81, 2, 279, 280, 281, 1, 3, 282, 101], [57, 4, 283, 81, 2, 284, 143, 1, 3, 101, 14, 4, 285, 2, 286, 143], [56, 4, 176, 14, 177, 178, 1, 3, 179, 144, 101], [9, 4, 180, 181], [58, 4, 180, 181], [121, 81, 74, 16, 82, 11, 102, 182, 2, 3, 287, 288], [121, 81, 59, 52, 11, 102, 182, 2, 3, 173, 174], [58, 4, 81, 8, 4, 289, 144, 81, 183], [121, 290, 4, 179, 144, 291, 81, 183, 9], [26, 3, 60], [184, 1, 3, 185, 1, 292], [184, 1, 3, 185, 1, 293, 294], [35, 21, 103, 122, 295], [35, 21, 103, 122, 296, 297, 2, 60], [35, 21, 103, 122, 145, 298, 1, 95, 299], [35, 21, 103, 122, 300, 2, 60, 1, 95, 8, 186], [48, 10, 53, 21, 123, 1, 3, 60, 83], [74, 146, 48, 10, 53, 21, 103, 1, 3, 60, 83], [74, 146, 48, 10, 53, 21, 187, 1, 3, 60, 32, 75, 301, 83], [74, 146, 48, 10, 53, 21, 123, 1, 3, 60, 32, 75, 302, 83], [48, 10, 53, 21, 187, 1, 3, 60, 32, 75, 147,

In [183]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(features)
features = scaler.fit_transform(features)
print(features[377])

[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -0.05150262 -0.05804832
 -0.07285687 -0.07404206 -0.05444687 -0.05518385 -0.07896221 -0.08928876
 -0.09772127 -0.09384884 -0.15412468 -0.17538823  0.733563   -0.14933617
 

In [181]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
unique_intent = list(leyzer_df['intent'])
print(unique_intent)
token = Tokenizer(1000)
token.fit_on_texts(unique_intent)
intent_2_index = token.texts_to_sequences(unique_intent)
labels = OneHotEncoder(sparse=False).fit_transform(intent_2_index)
print(labels)
label_2 = LabelEncoder().fit_transform(leyzer_df['intent'])
print(label_2)

['TurnOff', 'TurnOff', 'TurnOn', 'TurnOn', 'ChangeTemperature', 'SetTemperatureToValue', 'SetTemperatureToValueOnDevice', 'SetTemperatureBetweenValues', 'SetTemperatureOnDevice', 'GetHumidity', 'GetHumidity', 'GetTemperature', 'GetTemperature', 'GetTemperatureFromDevice', 'GetTemperatureFromDevice', 'OpenCalendar', 'CheckCalendarOnDate', 'CheckCalendarOnDate', 'AddEventWithName', 'AddEventWithName', 'AddEventOnDateWithDate', 'AddEventOnDateWithDate', 'NotifyOnEventStart', 'NotNotifyOnEventStart', 'NotNotifyOnEventInLocation', 'NotNotifyOnEventInLocation', 'NotifyOnEventInLocation', 'NotifyOnEventInLocation', 'NotifyOnEventInLocation', 'OpenContacts', 'OpenMyContact', 'EditContactWithName', 'EditContactWithName', 'EditContactWithName', 'EditContactWithNumber', 'EditNumberInContactWithName', 'EditNumberInContactWithNameToNumberAndToType', 'ShowContactWithEmail', 'ShowContactWithEmail', 'ShowContactWithEmail', 'ShowContactWithName', 'ShowContactWithName', 'ShowContactWithName', 'ShowConta

In [176]:
tokenizer.word_index

{'on': 1,
 'to': 2,
 'my': 3,
 'the': 4,
 'with': 5,
 'a': 6,
 'from': 7,
 'of': 8,
 'show': 9,
 'me': 10,
 'in': 11,
 'new': 12,
 'slack': 13,
 'and': 14,
 'number': 15,
 'i': 16,
 '555': 17,
 'create': 18,
 'contact': 19,
 'spotify': 20,
 'an': 21,
 'for': 22,
 'email': 23,
 'playlist': 24,
 'play': 25,
 'open': 26,
 'google': 27,
 'emails': 28,
 'post': 29,
 'translation': 30,
 'youtube': 31,
 'at': 32,
 'get': 33,
 'send': 34,
 'add': 35,
 'as': 36,
 'that': 37,
 'message': 38,
 'instagram': 39,
 'restaurants': 40,
 'contacts': 41,
 'name': 42,
 'track': 43,
 'italian': 44,
 'display': 45,
 'translate': 46,
 'french': 47,
 'notify': 48,
 '1': 49,
 'this': 50,
 'by': 51,
 'it': 52,
 'when': 53,
 'picture': 54,
 'english': 55,
 'change': 56,
 'set': 57,
 'check': 58,
 'is': 59,
 'calendar': 60,
 'using': 61,
 'drive': 62,
 'pictures': 63,
 'polish': 64,
 'twitter': 65,
 'call': 66,
 'subject': 67,
 'facebook': 68,
 'channel': 69,
 'speaker': 70,
 'song': 71,
 'tweets': 72,
 'matching

In [177]:
vocabulary = len(tokenizer.word_index)
print("Total numbers of vocabulary:", vocabulary)

Total numbers of vocabulary: 861


## Git_test_set

In [160]:
leyzer_test_df = pd.DataFrame(pd.read_csv("https://raw.githubusercontent.com/cartesinus/leyzer/master/corpora/0.1.0/leyzer-test-en-US-0.1.0.tsv", sep="\t"))
print(len(leyzer_test_df[['intent']].intent.unique()))
# print(leyzer_test_df[['intent']].intent.unique())

154


In [161]:
utr_test_tokenized = list(leyzer_test_df['utterance'].apply(lambda sent: word_tokenize(sent)))
print(utr_test_tokenized)

test_tokenizer = Tokenizer(1000)
test_tokenizer.fit_on_texts(utr_test_tokenized)
test_utr = test_tokenizer.texts_to_sequences(utr_test_tokenized)
print(test_utr)
test_sen = pad_sequences(test_utr, maxlen = 100)
print(test_sen[0])

t_scaler = StandardScaler().fit(test_sen)
test_sen = scaler.fit_transform(test_sen)

[['change', 'the', 'minimum', 'and', 'maximum', 'temperatures', 'on', 'my', 'thermostat'], ['change', 'the', 'temperature', 'on', 'my', 'thermostat'], ['check', 'the', 'humidity'], ['check', 'the', 'humidity', 'level'], ['get', 'the', 'humidity'], ['get', 'the', 'humidity', 'level'], ['show', 'the', 'humidity'], ['show', 'the', 'humidity', 'level'], ['what', 'temperature', 'do', 'I', 'have', 'in', 'here', 'according', 'to', 'my', 'heating', '?'], ['what', 'temperature', 'do', 'we', 'have', 'in', 'here', 'according', 'to', 'my', 'ac', '?'], ['what', 'temperature', 'is', 'it', 'in', 'here', 'according', 'to', 'my', 'aircon', '?'], ['check', 'the', 'temperature', 'of', 'the', 'living', 'room', 'room', 'temperature', 'sensor'], ['check', 'what', 'the', 'kitchen', 'home', 'temperature', 'sensor', 'says'], ['what', 'does', 'the', 'hall', 'indoor', 'temperature', 'sensor', 'say', '?'], ['what', 'does', 'the', 'bedroom', 'indoor', 'temperature', 'sensor', 'show', '?'], ['set', 'the', 'low', 't

In [164]:
unique_test_intent = list(set(leyzer_test_df['intent']))
print(unique_test_intent)
t_token = Tokenizer(1000)
t_token.fit_on_texts(unique_test_intent)
t_intent_2_index = t_token.texts_to_sequences(unique_test_intent)
label_t = LabelEncoder().fit_transform(leyzer_test_df['intent'])
print(label_t)

['OpenSlack', 'FindQueryOnChannel', 'ShowEmailWithPriority', 'SendMessageToChannel', 'ShowAlbumWithName', 'FollowUser', 'CreateFile', 'SetTemperatureToValue', 'CheckCalendarOnDate', 'ShowContactWithName', 'ShowStepsOnDate', 'DownloadAsPdf', 'SearchImagesWithTextOnEngineWithWidthAndWithHeightAndWithCondition', 'NotifyonNewOnChannel', 'SendMessageToUser', 'NextSong', 'RandomPage', 'ScrollDown', 'GetTemperature', 'ShowContactWithEmail', 'SendEmail', 'ShowSMS', 'SearchTextOnEngine', 'SearchByCategory', 'SearchByQuery', 'TurnOff', 'CreateFileWithName', 'SunriseAndSunsetInLocationAndOnDate', 'NotifyOnEventInLocation', 'SearchByReviewCountInLocation', 'SearchRestaurant', 'IncreaseVolume', 'MuteOn', 'NotifyOnWeight', 'FindChannelWithQuery', 'OpenMyContact', 'ShowContactWithNameAndWithNumberAndWithType', 'WeatherInLocation', 'OpenRestaurants', 'DetectLanguage', 'OpenFacebook', 'ShowBMI', 'OpenRestaurantsInLocation', 'SunriseAndSunsetInLocation', 'ShowPicturesWithFilter', 'OpenGdrive', 'PostPict

# Model

In [166]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [204]:
# X_train = pd.DataFrame(data = features)
# # print(X_train)
# y_train = pd.DataFrame(data = label_2)
# # print(y_train)
# X_test = pd.DataFrame(data = test_sen)
# y_test = pd.DataFrame(data = label_t)

X_train, X_test, y_train, y_test = train_test_split(features, label_2, test_size = 0.2)
clf = SVC(kernel='rbf', gamma=0.01, C=1000)
clf.fit(X_train, y_train)

prediction = clf.predict(X_test)
print(accuracy_score(y_test, prediction))

0.2894736842105263


In [206]:
from sklearn.ensemble import RandomForestClassifier

clf=RandomForestClassifier()
clf.fit(X_train, y_train);
prediction = clf.predict(X_test);
print(accuracy_score(y_test, prediction))

0.34210526315789475
